# Reti Domotica Anomaly Detection - Team 1

## Componenti del Team

- Francesco CATANIA
- Giorgio DEIANA
- Marco SASSO

## Requisiti di sistema

- Installare le librerie indicate nel file "requirements.txt"
- Installare ODBC Driver for SQL Server (https://docs.microsoft.com/it-it/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15)
- All'interno del file "settings.json" inserire le seguenti stringhe: "python.pythonPath": "{NOMEAMBIENTE}\\Scripts\\python.exe",
    "python.linting.pylintPath": "{NOMEAMBIENTE}\\bin\\pylint"

## Note Metodologiche

#### Modello adottato e addestramento

L'algoritmo di Machine Learning utilizzato per la rilevazione delle anomalie è l'Isolation Forest implementato all'interno della libreria Scikit-Learn. Si è scelto di addestrare cinque differenti modelli di Machine Learning - uno per ciascuna unità di misura rilevata ('ppm', '%', 'C°', 'W', 'Wh') - al fine di separare opportunamente i processi di ricerca di eventuali anomalie a seconda delle tipologie di rilevazioni presenti nel dataset "log". Dal progetto sono state esclusi i dati di tipo booleano riguardanti la rilevazione delle presenze.

I modelli sopracitati sono stati addestrati eseguendo un apposito notebook Python all'interno della piattaforma Databricks di Microsoft Azure, consultabile nella directory *Notebooks* della repository GitHub. I dati per l'addestramento sono stati estratti dal database SQL *CampusData* presente tra le risorse della sottoscrizione Microsoft Azure del corso.

Per il training di ciascun modello sono state considerate features in grado di cogliere il valore, la tipologia e l'eventuale stagionalità delle rilevazioni effettuate dai sensori del Campus, nello specifico:

- mese della rilevazione (derivata dal campo *Data* della tabella *dbo.Log* del database SQL)
- giorno della settimana della rilevazione (derivata dal campo *Data* del dataset *dbo.Log* del database SQL)
- valore della rilevazione (feature *Value* della tabella "dbo.Log" del database SQL)
- tipologia di rilevazione (feature *Id* della tabella *dbo.Group* del database SQL, ottenuta tramite join con la tabella *dbo.Log*)

#### Piattaforma Streamlit

L'utente può interagire con le funzionalità implementate nel progetto attraverso un'applicazione web realizzata in Streamlit che permette di:

- caricare nuovi dati dal database SQL Azure - diversi da quelli utilizzati per l'addestramento dei modelli - per effettuare delle nuove predizioni volte ad individuare eventuali anomalie;
- simulare una rilevazione di un sensore indicando la data, la tipologia di dato e il valore numerico per determinare, grazie alle predizioni dei modelli di Machine Learning, se si tratta di un'anomalia o meno.

Per l'accesso all'applicazione:
1. da terminale, digitare ***cd RetiDomoticaAnomalyDetect\DomoticaStreamLit*** per posizionarsi all'interno della directory *DomoticaStreamLit*
2. sempre da terminale, digitare ***streamlit run app.py*** per lanciare l'applicazione

La directory DomoticaStreamLit contiene il file app.py che lancia la definizione del corpo della dashboard definita nel file side_menu.py che si trova nella directory "DomoticaStreamLit/src/menu".
In side_menu.py recupero gli Id di buildin.csv e group.csv presenti nella directory "DomoticaStreamLit/dataset".
Il dataset group.csv è stato "processato" in modo da poter recuperare solo le descrizioni che sono state usate per il training dei modelli. 

All'interno della "sidebar" troviamo la selezione dell'Id del building(elenco degli edifici) per il quale è stato utilizzato un input di tipo "radio", avendo solo 3 scelte da elencare, invece per la scelta delle descrizioni in group.csv è stata generata una selectbox che contiene le descrizioni come valore visualizzato legato al relativo Id. Queste due scelte serviranno sia per la predizione utilizzando i dati da db sia i dati inseriti dall'utente. Per i dati scaricati da db la scelta di "Edificio" e "Descrizione" farà in modo che solo i dati relativi verranno simulati dal modello.
Sempre nella sidebar troviamo un oggetto di tipo "slider" per fare selezionare un valore all'utente e un oggetto tipo "date_input" per la scelta del giorno da inserire nella simulazione.

Nel corpo centrale della dashboard abbiamo 2 sezioni di scelta. La prima è quella relativa allo scaricamento dei dati da db azure, mentre la seconda è legata ai dati inseriti dall'utente. In entrambe le sezioni si può scegliere se effettuare la predizione con la visualizzazione del risultato a video, oppure effettuare la predizione e salvare il risultato su azure storage in formato json.
Nel caso di scelta di visualizzazione della predizione dei dati da db(senza salvataggio) verrà anche visualizzato un grafo prodotto utilizzando la libreria "altair."

La directory DomoticaStreamLit/src/data contiene il file load.py che espone diverse funzioni:
    - prelevare i dai da db azure
    - prelevare i modelli da azure
    - caricare i modelli in formato pkl presenti in locale

La directory DomoticaStreamLit/src/feature/ contine il file transform.py che espone le funzioni per il processamento del dataset di training e la chiamata della funzione "IsolationForest" per il training del modello. 

La directory DomoticaStreamLit/src/model contiene il file prediction.py che espone le seguenti funzioni:
    - forest_prediction : richiama la funzione predizione da remoto verso il modello salvato su azure
    - prediction_pkl : richiama la funzione di predizione sul modello caricato in locale

La directory DomoticaStreamLit/src/save contiene il file OperationsStorageAccount.py che espone diverse funzioni di salvataggio o richiesta dati da azure tra lui la funzione usata dall' application per salvare i dati sullo storage azure "sorageaccountgdeiana":
 - saveJsonStorageAccountFromDataframe a cui passiamo il datasets da salvare in formato json, il nome del container e l'unità di         misura dei dati su cui è stata fatta la predizione, unità di misura usata come parte del nome del file json salvato in remoto su      azure 

 





#### Salvataggio delle predizioni su Azure Storage

Le predizioni ottenute via Streamlit secondo le due modalità illustrate nel paragrafo precedente vengono archiviate in formato JSON all'interno di un apposito Azure Storage denominato *sorageaccountgdeiana*. 
Nello specifico, le predizioni frutto della lettura da database SQL Azure vengono salvate nel container *predictcontainerdomoticaadsqldb*, mentre quelle generate dalla simulazione dell'utente vengono archiviate all'interno del container *predictcontainerdomoticaaduser*.



## Navigazione cartelle della repository GitHub

Le varie componenti del progetto sono organizzate secondo la seguente struttura di cartelle:

- **Datasets** contiene i file *building.csv*, *group.csv* e *log2.csv* messi a disposizione a inizio progetto per le analisi preliminari in locale;


- **DomoticaStreamLit** contiene il file Python *app.py* per il lancio dell'applicazione web in Streamlit; 


- **Notebooks** contiene i notebook utilizzati per l'esplorazione preliminare del dataset e per il training dei modelli in Databricks; 


- **TeamLibraries** contiene le funzioni Python sviluppate internamente dal team, suddivise in tre diversi moduli:
    - *LoadData.py* racchiude le funzioni per la connessione al database SQL Azure, lo scarico dei dati e il caricamento dei modelli versionati in Databricks;
    
    - *ModelFunctions.py* comprende le funzioni per il preprocessing dei dati e la predizione tramite i modelli;
    
    - *OperationsStorageAccount.py* contiene le funzioni per la connessione allo Storage Account Azure e il salvataggio delle predizioni in formato JSON negli appositi container;
    
    ***A causa di imprevisti tecnici che si sono verificati con il serving dei modelli in Databricks, la cartella TeamLibraries contiene un backup di tutti i modelli in formato Pickle addestrati in locale***.
